In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 10.2MB/s 
     |████████████████████████████████| 3.0MB 52.2MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
     |████████████████████████████████| 1.1MB 44.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3930fb2f59b3a8fb20fac35c1404094f24b02559f4f03c3e847ba6e1cb77f4fa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
output_dir = '/content/gdrive/My Drive/stackoverflow/weights/'
model = T5ForConditionalGeneration.from_pretrained(output_dir)
tokenizer = T5Tokenizer.from_pretrained(output_dir)

In [ ]:

import os
import time
import datetime
import torch
from torch.utils.data import Dataset



import argparse
import logging
import os
import random

import numpy as np

import torch
from torch.utils.data import DataLoader

from transformers import (
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)


In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import tensorflow as tf
train_dataset_fp = tf.keras.utils.get_file('/content/gdrive/My Drive/stackoverflow/summarization_data.csv','summarization_data.csv')
batch_size = 8

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    select_columns = ['title','body'],
    num_epochs=1)

In [ ]:
def encode_file(tokenizer,data,max_length,return_tensors="pt"):
    examples = []
    for sen in data:
       tokenized = tokenizer.batch_encode_plus(
                [sen], max_length=max_length, truncation=True,padding ='max_length', return_tensors=return_tensors)
       examples.append(tokenized)
    return examples

In [ ]:
model = model.to(device)

In [ ]:
num_train_epochs = 2
#model.resize_token_embeddings(len(tokenizer))
weight_decay =0.0
learning_rate = 1e-4 
adam_epsilon = 1e-8
warmup_steps = 0
t_total = (607276 // 8 ) * float(2)

In [ ]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
    )
optimizer.load_state_dict(torch.load(os.path.join(output_dir, 'optimizer.pt')))
scheduler.load_state_dict(torch.load(os.path.join(output_dir, 'scheduler.pt')))

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:218: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
epochs = 5
for epoc in range(epochs):
  t0 = time.time()
  print("")
  print('======== Epoch {} ========'.format(epoc+1))
  model.train()
  total_train_loss = 0
  for i,batch in enumerate(train_dataset):
    title = []
    body = []
    for item in batch['title'].numpy():
      title.append(item.decode('utf-8'))
    for item in batch['body'].numpy():
      body.append(item.decode('utf-8')) 
    encoded_title = encode_file(tokenizer,title,120)
    encoded_body = encode_file(tokenizer,body,512)
    ids = torch.stack([x["input_ids"] for x in encoded_body]).squeeze()# BS x SL
    mask = torch.stack([x["attention_mask"] for x in encoded_body]).squeeze() # BS x SL
    pad_token_id = tokenizer.pad_token_id
    y = torch.stack([x["input_ids"] for x in encoded_title]).squeeze()
    y = y.to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = ids.to(device, dtype = torch.long)
    mask = mask.to(device, dtype = torch.long)
    model.zero_grad()
    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
    loss = outputs[0]
    total_train_loss += loss.item()
    
    if i%500 == 0:
      print("batch :" + str(i)+ "Training Loss: " +str(total_train_loss))
    if i % 20000 == 0:
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
        torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
  training_time = format_time(time.time() - t0)
  avg_train_loss = total_train_loss / 1050994
  print("traing loss epoch"+str(epoc+1)+":"+str(avg_train_loss))
  print("time : {}".format(training_time))
  
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
  torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))


======== Epoch 1 ========


/usr/local/lib/python3.6/dist-packages/transformers/modeling_t5.py:1128: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


batch :0Training Loss: 5.187439918518066


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


batch :500Training Loss: 1533.8602048158646
batch :1000Training Loss: 2913.984565615654
batch :1500Training Loss: 4241.05873811245
batch :2000Training Loss: 5567.6087691783905
batch :2500Training Loss: 6862.107839345932
batch :3000Training Loss: 8147.294431686401
batch :3500Training Loss: 9435.012337207794
batch :4000Training Loss: 10705.89479315281
batch :4500Training Loss: 11972.306923747063
batch :5000Training Loss: 13214.92782485485
batch :5500Training Loss: 14471.89020550251
batch :6000Training Loss: 15712.807140946388
batch :6500Training Loss: 16954.91726064682
batch :7000Training Loss: 18192.134781599045
batch :7500Training Loss: 19419.502864599228
batch :8000Training Loss: 20651.13777422905
batch :8500Training Loss: 21884.291428804398
batch :9000Training Loss: 23108.809393525124
batch :9500Training Loss: 24329.61497926712
batch :10000Training Loss: 25555.460893154144
batch :10500Training Loss: 26775.082923173904
batch :11000Training Loss: 28004.104807019234
batch :11500Training

In [ ]:
epochs = 5
for epoc in range(epochs):
  t0 = time.time()
  print("")
  print('======== Epoch {} ========'.format(epoc+1))
  model.train()
  total_train_loss = 0
  for i,batch in enumerate(train_dataset):
    title = []
    body = []
    for item in batch['title'].numpy():
      title.append(item.decode('utf-8'))
    for item in batch['body'].numpy():
      body.append(item.decode('utf-8')) 
    encoded_title = encode_file(tokenizer,title,120)
    encoded_body = encode_file(tokenizer,body,512)
    ids = torch.stack([x["input_ids"] for x in encoded_body]).squeeze()# BS x SL
    mask = torch.stack([x["attention_mask"] for x in encoded_body]).squeeze() # BS x SL
    pad_token_id = tokenizer.pad_token_id
    y = torch.stack([x["input_ids"] for x in encoded_title]).squeeze()
    y = y.to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = ids.to(device, dtype = torch.long)
    mask = mask.to(device, dtype = torch.long)
    model.zero_grad()
    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
    loss = outputs[0]
    total_train_loss += loss.item()
    
    if i%500 == 0:
      print("batch :" + str(i)+ "Training Loss: " +str(total_train_loss))

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
  training_time = format_time(time.time() - t0)
  avg_train_loss = total_train_loss / 1050994
  print("traing loss epoch"+str(epoc+1)+":"+str(avg_train_loss))
  print("time : {}".format(training_time))
  
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
  torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))


======== Epoch 1 ========


/usr/local/lib/python3.6/dist-packages/transformers/modeling_t5.py:1128: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


batch :0Training Loss: 5.295326232910156
batch :500Training Loss: 1528.4088693857193
batch :1000Training Loss: 2921.5784620046616
batch :1500Training Loss: 4270.449675321579
batch :2000Training Loss: 5579.342727184296
batch :2500Training Loss: 6871.840752840042
batch :3000Training Loss: 8162.124095320702
batch :3500Training Loss: 9444.195762991905
batch :4000Training Loss: 10714.794030189514
batch :4500Training Loss: 11988.838445544243
batch :5000Training Loss: 13228.565332174301
batch :5500Training Loss: 14484.114659190178
batch :6000Training Loss: 15722.980154633522
batch :6500Training Loss: 16950.769930958748
batch :7000Training Loss: 18169.60313117504
batch :7500Training Loss: 19406.309152126312
batch :8000Training Loss: 20632.255396723747
batch :8500Training Loss: 21857.136608600616
batch :9000Training Loss: 23089.353440523148
batch :9500Training Loss: 24307.540618538857
batch :10000Training Loss: 25543.613577723503
batch :10500Training Loss: 26782.20229613781
batch :11000Training

In [ ]:
epochs = 5
for epoc in range(epochs):
  t0 = time.time()
  print("")
  print('======== Epoch {} ========'.format(epoc+1))
  model.train()
  total_train_loss = 0
  for i,batch in enumerate(train_dataset):
    title = []
    body = []
    for item in batch['title'].numpy():
      title.append(item.decode('utf-8'))
    for item in batch['body'].numpy():
      body.append(item.decode('utf-8')) 
    encoded_title = encode_file(tokenizer,title,120)
    encoded_body = encode_file(tokenizer,body,512)
    ids = torch.stack([x["input_ids"] for x in encoded_body]).squeeze()# BS x SL
    mask = torch.stack([x["attention_mask"] for x in encoded_body]).squeeze() # BS x SL
    pad_token_id = tokenizer.pad_token_id
    y = torch.stack([x["input_ids"] for x in encoded_title]).squeeze()
    y = y.to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = ids.to(device, dtype = torch.long)
    mask = mask.to(device, dtype = torch.long)
    model.zero_grad()
    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
    loss = outputs[0]
    total_train_loss += loss.item()
    
    if i%500 == 0:
      print("batch :" + str(i)+ "Training Loss: " +str(total_train_loss))

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
  training_time = format_time(time.time() - t0)
  avg_train_loss = total_train_loss / 1050994
  print("traing loss epoch"+str(epoc+1)+":"+str(avg_train_loss))
  print("time : {}".format(training_time))
  
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
  torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))


======== Epoch 1 ========


/usr/local/lib/python3.6/dist-packages/transformers/modeling_t5.py:1128: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


batch :0Training Loss: 2.044417142868042
batch :500Training Loss: 1007.9309905767441
batch :1000Training Loss: 2018.7188791036606
batch :1500Training Loss: 3046.364337205887
batch :2000Training Loss: 4070.3735456466675
batch :2500Training Loss: 5108.851502358913
batch :3000Training Loss: 6130.290075838566
batch :3500Training Loss: 7154.580143868923
batch :4000Training Loss: 8181.764933288097
batch :4500Training Loss: 9219.019903123379
batch :5000Training Loss: 10246.710900008678
batch :5500Training Loss: 11280.93491858244
batch :6000Training Loss: 12296.160244166851
batch :6500Training Loss: 13318.473236382008
batch :7000Training Loss: 14340.052779138088
batch :7500Training Loss: 15373.599617660046
batch :8000Training Loss: 16413.39070022106
batch :8500Training Loss: 17435.94557440281
batch :9000Training Loss: 18476.59170806408
batch :9500Training Loss: 19514.4249856472
batch :10000Training Loss: 20545.569168686867
batch :10500Training Loss: 21600.828709363937
batch :11000Training Loss

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



======== Epoch 2 ========
batch :0Training Loss: 1.7710051536560059
batch :500Training Loss: 1000.5923355817795
batch :1000Training Loss: 1994.8049125671387
batch :1500Training Loss: 3016.090177476406
batch :2000Training Loss: 4016.832619905472
batch :2500Training Loss: 5041.817929983139
batch :3000Training Loss: 6059.141159415245
batch :3500Training Loss: 7070.916650772095
batch :4000Training Loss: 8093.582837104797
batch :4500Training Loss: 9119.4923671484
batch :5000Training Loss: 10142.546251773834
batch :5500Training Loss: 11158.155492782593
batch :6000Training Loss: 12163.843958437443
batch :6500Training Loss: 13180.03725939989
batch :7000Training Loss: 14198.459120512009
batch :7500Training Loss: 15225.758615851402
batch :8000Training Loss: 16245.111999809742
batch :8500Training Loss: 17267.669735491276
batch :9000Training Loss: 18285.899792850018
batch :9500Training Loss: 19313.707645833492
batch :10000Training Loss: 20336.421181499958
batch :10500Training Loss: 21366.74151796

In [ ]:
epochs = 5
for epoc in range(epochs):
  t0 = time.time()
  print("")
  print('======== Epoch {} ========'.format(epoc+1))
  model.train()
  total_train_loss = 0
  for i,batch in enumerate(train_dataset):
    title = []
    body = []
    for item in batch['title'].numpy():
      title.append(item.decode('utf-8'))
    for item in batch['body'].numpy():
      body.append(item.decode('utf-8')) 
    encoded_title = encode_file(tokenizer,title,120)
    encoded_body = encode_file(tokenizer,body,512)
    ids = torch.stack([x["input_ids"] for x in encoded_body]).squeeze()# BS x SL
    mask = torch.stack([x["attention_mask"] for x in encoded_body]).squeeze() # BS x SL
    pad_token_id = tokenizer.pad_token_id
    y = torch.stack([x["input_ids"] for x in encoded_title]).squeeze()
    y = y.to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = ids.to(device, dtype = torch.long)
    mask = mask.to(device, dtype = torch.long)
    model.zero_grad()
    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
    loss = outputs[0]
    total_train_loss += loss.item()
    
    if i%500 == 0:
      print("batch :" + str(i)+ "Training Loss: " +str(total_train_loss))
    if i % 20000 == 0:
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
        torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
  training_time = format_time(time.time() - t0)
  avg_train_loss = total_train_loss / 1050994
  print("traing loss epoch"+str(epoc+1)+":"+str(avg_train_loss))
  print("time : {}".format(training_time))
  
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
  torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))


======== Epoch 1 ========


/usr/local/lib/python3.6/dist-packages/transformers/modeling_t5.py:1128: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


batch :0Training Loss: 2.553109884262085


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


batch :500Training Loss: 988.6108185052872
batch :1000Training Loss: 2005.3290561437607
batch :1500Training Loss: 3015.828029513359
batch :2000Training Loss: 4015.1137213110924
batch :2500Training Loss: 5047.26597648859
batch :3000Training Loss: 6058.422338426113
batch :3500Training Loss: 7077.438912451267
batch :4000Training Loss: 8089.76018935442
batch :4500Training Loss: 9113.382313072681
batch :5000Training Loss: 10126.40777951479
batch :5500Training Loss: 11147.421782910824
batch :6000Training Loss: 12163.933211863041
batch :6500Training Loss: 13192.830403149128
batch :7000Training Loss: 14213.68697053194
batch :7500Training Loss: 15222.291413486004
batch :8000Training Loss: 16257.561263740063
batch :8500Training Loss: 17281.995755016804
batch :9000Training Loss: 18309.77070862055
batch :9500Training Loss: 19328.594387590885
batch :10000Training Loss: 20358.44654637575
batch :10500Training Loss: 21383.82090872526
batch :11000Training Loss: 22395.991100132465
batch :11500Training L